In [3]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os, glob
from functools import reduce
import operator
import math
from math import pi

In [11]:
# bouy_data = pd.read_csv('/home/ly/xia_datan/bouydataTrace/data/bouydata_4_hycom.csv')
# bouy_data = pd.read_csv('/home/ly/xia_datan/bouydataTrace/data/bouydata_3_1.csv')
bouy_data = pd.read_csv('bouydata_3_1.csv')
numbers_pred = 12 #点数 # 数据时间分辨率 6小时  train_x \\ 用12 * 6 小时数据预报
forecast_hour = 24 #时间 # 预测时间序列   预报4个点（时间分辨率为6 hours）
bouy_data

,id,time,latitude,longitude,sst,VE,VN,SPD,VAR.LAT,VAR.LON,VAR.TEMP
0,107601,2014-12-22 06:00:00,22.547,124.965,24.658,-24.346,45.222,51.359,0.000009,0.000020,0.001724
1,107601,2014-12-22 12:00:00,22.657,124.938,24.485,-3.633,66.491,66.590,0.000004,0.000008,0.001584
2,107601,2014-12-22 18:00:00,22.807,124.949,24.320,-9.418,62.888,63.589,0.000006,0.000012,0.001869
3,107601,2014-12-23 00:00:00,22.902,124.898,24.148,-36.169,53.911,64.920,0.000004,0.000008,0.001665
4,107601,2014-12-23 06:00:00,23.017,124.797,24.101,-37.589,58.931,69.898,0.000009,0.000019,0.001918
...,...,...,...,...,...,...,...,...,...,...,...
69468,300534064920860,2023-10-13 12:00:00,21.221,111.331,27.962,-2.695,3.717,4.592,0.000002,0.000004,0.001684
69469,300534064920860,2023-10-13 18:00:00,21.219,111.327,27.754,-5.732,0.897,5.801,0.000002,0.000004,0.001684
69470,300534064920860,2023-10-14 00:00:00,21.225,111.308,27.740,-5.591,0.628,5.626,0.000002,0.000004,0.001684
69471,300534064920860,2023-10-14 06:00:00,21.221,111.304,27.896,-1.006,2.758,2.936,0.000002,0.000004,0.001684


In [12]:
# 算距离
def getDistance(latA, lonA, latB, lonB):
    ra = 6378136.49  # radius of equator: meter
    rb = 6356755  # radius of polar: meter

    flatten = (ra - rb) / ra  # Partial rate of the earth
    # change angle to radians
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)
 
    pA = math.atan(rb / ra * math.tan(radLatA))
    pB = math.atan(rb / ra * math.tan(radLatB))
    if radLonA == radLonB:
        x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(0.001))
    else:
        x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(radLonA - radLonB))
    c1 = (math.sin(x) - x) * (math.sin(pA) + math.sin(pB))**2 / math.cos(x / 2)**2
    c2 = (math.sin(x) + x) * (math.sin(pA) - math.sin(pB))**2 / math.sin(x / 2)**2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)/1000
    
    dLon = radLonB - radLonA
    yy = math.sin(dLon) * math.cos(radLatB)
    xx = math.cos(radLatA) * math.sin(radLatB) - math.sin(radLatA) * math.cos(radLatB) * math.cos(dLon)
    brng = math.degrees(math.atan2(yy, xx))
    brng = (brng + 180) % 360
#     if brng>180:
#         brng=brng-360
#     brng=brng*math.pi/180
    return distance,brng

def remove_duplicates(input_list):
    new_list = []
    for element in input_list:
        if element not in new_list:
            new_list.append(element)
    return new_list    

def shift(data,forcast_hour,numbers_pred):
    cols=[]
    # lat_data=data['latitude']
    # lon_data=data['longitude']
    # m=int(forcast_hour/3)+8
    # n=int(forcast_hour/3)-1
    # numbers_pred = 12 
    # forecast_hour = 24
    m=int(forcast_hour/6)+numbers_pred
    n=int(forcast_hour/6)-1
    for i in range(m,n,-1):
        cols.append(data.shift(i))      
    
    for i in range(n,-1,-1):
        cols.append(data.shift(i))
    # for i in range(n,-1,-1):
    #     cols.append(data.shift(i))        
    agg = pd.concat(cols, axis=1)
    essemble=agg.dropna(axis=0,how='any')
    return essemble    

In [13]:
clean_id = remove_duplicates(bouy_data["id"])
index_i = []
for i in range(len(clean_id)):
    indeices = bouy_data[bouy_data["id"] == clean_id[i]].index
    index_i.append(indeices[0])
data_track = bouy_data
print(len(index_i))

360


In [14]:
essemble_test = []
for i in range(len(index_i)):
# for i in range(5,6):
    print(i)
    m = index_i[i]
    if i == len(index_i)-1:
        n = len(data_track)
    else:
        n = index_i[i+1]
    data = data_track[m:n]
    # print(len(data))
    dataframe_trace = pd.DataFrame(data)
    essemble = shift(dataframe_trace,forecast_hour,numbers_pred)
    print(len(essemble))
    print("----------------")
    essemble_value = essemble.values
    essemble_list = list(essemble_value)
    dif_data = [row[11:] for row in essemble_list]
    result = []
    for row in dif_data:
        new_row = []
        for j in range(0,len(row),11):
            new_row.extend(row[j:j+2])
        result.append(new_row)
        
    essemble_test.append(result)

0
21
----------------
1
168
----------------
2
43
----------------
3
41
----------------
4
39
----------------
5
116
----------------
6
108
----------------
7
109
----------------
8
66
----------------
9
343
----------------
10
108
----------------
11
43
----------------
12
167
----------------
13
156
----------------
14
7
----------------
15
221
----------------
16
111
----------------
17
6
----------------
18
6
----------------
19
96
----------------
20
132
----------------
21
40
----------------
22
41
----------------
23
20
----------------
24
102
----------------
25
260
----------------
26
281
----------------
27
19
----------------
28
25
----------------
29
268
----------------
30
159
----------------
31
46
----------------
32
3
----------------
33
62
----------------
34
312
----------------
35
197
----------------
36
50
----------------
37
39
----------------
38
571
----------------
39
268
----------------
40
15
----------------
41
49
----------------
42
31
----------------
43
17

In [15]:
aaa = pd.DataFrame(result)
aaa

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,3.005341e+14,2023-09-28 18:00:00,3.005341e+14,2023-09-29 00:00:00,3.005341e+14,2023-09-29 06:00:00,3.005341e+14,2023-09-29 12:00:00,3.005341e+14,2023-09-29 18:00:00,...,3.005341e+14,2023-10-01 12:00:00,3.005341e+14,2023-10-01 18:00:00,3.005341e+14,2023-10-02 00:00:00,3.005341e+14,2023-10-02 06:00:00,300534064920860,2023-10-02 12:00:00
1,3.005341e+14,2023-09-29 00:00:00,3.005341e+14,2023-09-29 06:00:00,3.005341e+14,2023-09-29 12:00:00,3.005341e+14,2023-09-29 18:00:00,3.005341e+14,2023-09-30 00:00:00,...,3.005341e+14,2023-10-01 18:00:00,3.005341e+14,2023-10-02 00:00:00,3.005341e+14,2023-10-02 06:00:00,3.005341e+14,2023-10-02 12:00:00,300534064920860,2023-10-02 18:00:00
2,3.005341e+14,2023-09-29 06:00:00,3.005341e+14,2023-09-29 12:00:00,3.005341e+14,2023-09-29 18:00:00,3.005341e+14,2023-09-30 00:00:00,3.005341e+14,2023-09-30 06:00:00,...,3.005341e+14,2023-10-02 00:00:00,3.005341e+14,2023-10-02 06:00:00,3.005341e+14,2023-10-02 12:00:00,3.005341e+14,2023-10-02 18:00:00,300534064920860,2023-10-03 00:00:00
3,3.005341e+14,2023-09-29 12:00:00,3.005341e+14,2023-09-29 18:00:00,3.005341e+14,2023-09-30 00:00:00,3.005341e+14,2023-09-30 06:00:00,3.005341e+14,2023-09-30 12:00:00,...,3.005341e+14,2023-10-02 06:00:00,3.005341e+14,2023-10-02 12:00:00,3.005341e+14,2023-10-02 18:00:00,3.005341e+14,2023-10-03 00:00:00,300534064920860,2023-10-03 06:00:00
4,3.005341e+14,2023-09-29 18:00:00,3.005341e+14,2023-09-30 00:00:00,3.005341e+14,2023-09-30 06:00:00,3.005341e+14,2023-09-30 12:00:00,3.005341e+14,2023-09-30 18:00:00,...,3.005341e+14,2023-10-02 12:00:00,3.005341e+14,2023-10-02 18:00:00,3.005341e+14,2023-10-03 00:00:00,3.005341e+14,2023-10-03 06:00:00,300534064920860,2023-10-03 12:00:00
5,3.005341e+14,2023-09-30 00:00:00,3.005341e+14,2023-09-30 06:00:00,3.005341e+14,2023-09-30 12:00:00,3.005341e+14,2023-09-30 18:00:00,3.005341e+14,2023-10-01 00:00:00,...,3.005341e+14,2023-10-02 18:00:00,3.005341e+14,2023-10-03 00:00:00,3.005341e+14,2023-10-03 06:00:00,3.005341e+14,2023-10-03 12:00:00,300534064920860,2023-10-03 18:00:00
6,3.005341e+14,2023-09-30 06:00:00,3.005341e+14,2023-09-30 12:00:00,3.005341e+14,2023-09-30 18:00:00,3.005341e+14,2023-10-01 00:00:00,3.005341e+14,2023-10-01 06:00:00,...,3.005341e+14,2023-10-03 00:00:00,3.005341e+14,2023-10-03 06:00:00,3.005341e+14,2023-10-03 12:00:00,3.005341e+14,2023-10-03 18:00:00,300534064920860,2023-10-04 00:00:00
7,3.005341e+14,2023-09-30 12:00:00,3.005341e+14,2023-09-30 18:00:00,3.005341e+14,2023-10-01 00:00:00,3.005341e+14,2023-10-01 06:00:00,3.005341e+14,2023-10-01 12:00:00,...,3.005341e+14,2023-10-03 06:00:00,3.005341e+14,2023-10-03 12:00:00,3.005341e+14,2023-10-03 18:00:00,3.005341e+14,2023-10-04 00:00:00,300534064920860,2023-10-04 06:00:00
8,3.005341e+14,2023-09-30 18:00:00,3.005341e+14,2023-10-01 00:00:00,3.005341e+14,2023-10-01 06:00:00,3.005341e+14,2023-10-01 12:00:00,3.005341e+14,2023-10-01 18:00:00,...,3.005341e+14,2023-10-03 12:00:00,3.005341e+14,2023-10-03 18:00:00,3.005341e+14,2023-10-04 00:00:00,3.005341e+14,2023-10-04 06:00:00,300534064920860,2023-10-04 12:00:00
9,3.005341e+14,2023-10-01 00:00:00,3.005341e+14,2023-10-01 06:00:00,3.005341e+14,2023-10-01 12:00:00,3.005341e+14,2023-10-01 18:00:00,3.005341e+14,2023-10-02 00:00:00,...,3.005341e+14,2023-10-03 18:00:00,3.005341e+14,2023-10-04 00:00:00,3.005341e+14,2023-10-04 06:00:00,3.005341e+14,2023-10-04 12:00:00,300534064920860,2023-10-04 18:00:00


In [16]:
essemble_ds = reduce(operator.add, essemble_test)
print(len(essemble_ds))

63713


In [17]:
bbb = pd.DataFrame(essemble_ds)
bbb

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,1.076010e+05,2014-12-22 12:00:00,1.076010e+05,2014-12-22 18:00:00,1.076010e+05,2014-12-23 00:00:00,1.076010e+05,2014-12-23 06:00:00,1.076010e+05,2014-12-23 12:00:00,...,1.076010e+05,2014-12-25 06:00:00,1.076010e+05,2014-12-25 12:00:00,1.076010e+05,2014-12-25 18:00:00,1.076010e+05,2014-12-26 00:00:00,107601,2014-12-26 06:00:00
1,1.076010e+05,2014-12-22 18:00:00,1.076010e+05,2014-12-23 00:00:00,1.076010e+05,2014-12-23 06:00:00,1.076010e+05,2014-12-23 12:00:00,1.076010e+05,2014-12-23 18:00:00,...,1.076010e+05,2014-12-25 12:00:00,1.076010e+05,2014-12-25 18:00:00,1.076010e+05,2014-12-26 00:00:00,1.076010e+05,2014-12-26 06:00:00,107601,2014-12-26 12:00:00
2,1.076010e+05,2014-12-23 00:00:00,1.076010e+05,2014-12-23 06:00:00,1.076010e+05,2014-12-23 12:00:00,1.076010e+05,2014-12-23 18:00:00,1.076010e+05,2014-12-24 00:00:00,...,1.076010e+05,2014-12-25 18:00:00,1.076010e+05,2014-12-26 00:00:00,1.076010e+05,2014-12-26 06:00:00,1.076010e+05,2014-12-26 12:00:00,107601,2014-12-26 18:00:00
3,1.076010e+05,2014-12-23 06:00:00,1.076010e+05,2014-12-23 12:00:00,1.076010e+05,2014-12-23 18:00:00,1.076010e+05,2014-12-24 00:00:00,1.076010e+05,2014-12-24 06:00:00,...,1.076010e+05,2014-12-26 00:00:00,1.076010e+05,2014-12-26 06:00:00,1.076010e+05,2014-12-26 12:00:00,1.076010e+05,2014-12-26 18:00:00,107601,2014-12-27 00:00:00
4,1.076010e+05,2014-12-23 12:00:00,1.076010e+05,2014-12-23 18:00:00,1.076010e+05,2014-12-24 00:00:00,1.076010e+05,2014-12-24 06:00:00,1.076010e+05,2014-12-24 12:00:00,...,1.076010e+05,2014-12-26 06:00:00,1.076010e+05,2014-12-26 12:00:00,1.076010e+05,2014-12-26 18:00:00,1.076010e+05,2014-12-27 00:00:00,107601,2014-12-27 06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63708,3.005341e+14,2023-10-09 18:00:00,3.005341e+14,2023-10-10 00:00:00,3.005341e+14,2023-10-10 06:00:00,3.005341e+14,2023-10-10 12:00:00,3.005341e+14,2023-10-10 18:00:00,...,3.005341e+14,2023-10-12 12:00:00,3.005341e+14,2023-10-12 18:00:00,3.005341e+14,2023-10-13 00:00:00,3.005341e+14,2023-10-13 06:00:00,300534064920860,2023-10-13 12:00:00
63709,3.005341e+14,2023-10-10 00:00:00,3.005341e+14,2023-10-10 06:00:00,3.005341e+14,2023-10-10 12:00:00,3.005341e+14,2023-10-10 18:00:00,3.005341e+14,2023-10-11 00:00:00,...,3.005341e+14,2023-10-12 18:00:00,3.005341e+14,2023-10-13 00:00:00,3.005341e+14,2023-10-13 06:00:00,3.005341e+14,2023-10-13 12:00:00,300534064920860,2023-10-13 18:00:00
63710,3.005341e+14,2023-10-10 06:00:00,3.005341e+14,2023-10-10 12:00:00,3.005341e+14,2023-10-10 18:00:00,3.005341e+14,2023-10-11 00:00:00,3.005341e+14,2023-10-11 06:00:00,...,3.005341e+14,2023-10-13 00:00:00,3.005341e+14,2023-10-13 06:00:00,3.005341e+14,2023-10-13 12:00:00,3.005341e+14,2023-10-13 18:00:00,300534064920860,2023-10-14 00:00:00
63711,3.005341e+14,2023-10-10 12:00:00,3.005341e+14,2023-10-10 18:00:00,3.005341e+14,2023-10-11 00:00:00,3.005341e+14,2023-10-11 06:00:00,3.005341e+14,2023-10-11 12:00:00,...,3.005341e+14,2023-10-13 06:00:00,3.005341e+14,2023-10-13 12:00:00,3.005341e+14,2023-10-13 18:00:00,3.005341e+14,2023-10-14 00:00:00,300534064920860,2023-10-14 06:00:00


In [18]:
# 判断数据抽没抽对
for i in range(len(essemble_ds)-1):
    time1 = essemble_ds[i][1]
    time2 = essemble_ds[i+1][1]
    
    # 将字符串转换为datetime对象
    datetime1 = datetime.strptime(time1, "%Y-%m-%d %H:%M:%S")
    datetime2 = datetime.strptime(time2, "%Y-%m-%d %H:%M:%S")
    
    # 计算时间间隔
    time_difference = datetime2 - datetime1
    
    # 将时间间隔转换为秒
    seconds_difference = time_difference.total_seconds()
    
    if essemble_ds[i][0] == essemble_ds[i+1][0]:
        if seconds_difference != 21600:
            print(i)
            print(essemble_ds[i])
            print(essemble_ds[i+1])
            print("----------------------------")

In [20]:
import csv
def essemble_path(name,result):
    pathname = "/home/ly/xia_datan/wangying/trace/_72_24/" + name
    with open(pathname,"w",newline="") as csvfile:
        writer = csv.writer(csvfile)
        for row in result:
            writer.writerow(row)
essemble_path(f'fore72_track_{numbers_pred*6}_{forecast_hour}_time.csv',essemble_ds)
# essemble_path("fore72_track_72_24_14parameters_nodiff_time.csv",essemble_ds)

In [21]:
import pandas as pd
data = pd.read_csv('/home/ly/xia_datan/wangying/trace/_72_24/fore72_track_72_24_time.csv',header=None)
data

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,1.076010e+05,2014-12-22 12:00:00,1.076010e+05,2014-12-22 18:00:00,1.076010e+05,2014-12-23 00:00:00,1.076010e+05,2014-12-23 06:00:00,1.076010e+05,2014-12-23 12:00:00,...,1.076010e+05,2014-12-25 06:00:00,1.076010e+05,2014-12-25 12:00:00,1.076010e+05,2014-12-25 18:00:00,1.076010e+05,2014-12-26 00:00:00,107601,2014-12-26 06:00:00
1,1.076010e+05,2014-12-22 18:00:00,1.076010e+05,2014-12-23 00:00:00,1.076010e+05,2014-12-23 06:00:00,1.076010e+05,2014-12-23 12:00:00,1.076010e+05,2014-12-23 18:00:00,...,1.076010e+05,2014-12-25 12:00:00,1.076010e+05,2014-12-25 18:00:00,1.076010e+05,2014-12-26 00:00:00,1.076010e+05,2014-12-26 06:00:00,107601,2014-12-26 12:00:00
2,1.076010e+05,2014-12-23 00:00:00,1.076010e+05,2014-12-23 06:00:00,1.076010e+05,2014-12-23 12:00:00,1.076010e+05,2014-12-23 18:00:00,1.076010e+05,2014-12-24 00:00:00,...,1.076010e+05,2014-12-25 18:00:00,1.076010e+05,2014-12-26 00:00:00,1.076010e+05,2014-12-26 06:00:00,1.076010e+05,2014-12-26 12:00:00,107601,2014-12-26 18:00:00
3,1.076010e+05,2014-12-23 06:00:00,1.076010e+05,2014-12-23 12:00:00,1.076010e+05,2014-12-23 18:00:00,1.076010e+05,2014-12-24 00:00:00,1.076010e+05,2014-12-24 06:00:00,...,1.076010e+05,2014-12-26 00:00:00,1.076010e+05,2014-12-26 06:00:00,1.076010e+05,2014-12-26 12:00:00,1.076010e+05,2014-12-26 18:00:00,107601,2014-12-27 00:00:00
4,1.076010e+05,2014-12-23 12:00:00,1.076010e+05,2014-12-23 18:00:00,1.076010e+05,2014-12-24 00:00:00,1.076010e+05,2014-12-24 06:00:00,1.076010e+05,2014-12-24 12:00:00,...,1.076010e+05,2014-12-26 06:00:00,1.076010e+05,2014-12-26 12:00:00,1.076010e+05,2014-12-26 18:00:00,1.076010e+05,2014-12-27 00:00:00,107601,2014-12-27 06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63708,3.005341e+14,2023-10-09 18:00:00,3.005341e+14,2023-10-10 00:00:00,3.005341e+14,2023-10-10 06:00:00,3.005341e+14,2023-10-10 12:00:00,3.005341e+14,2023-10-10 18:00:00,...,3.005341e+14,2023-10-12 12:00:00,3.005341e+14,2023-10-12 18:00:00,3.005341e+14,2023-10-13 00:00:00,3.005341e+14,2023-10-13 06:00:00,300534064920860,2023-10-13 12:00:00
63709,3.005341e+14,2023-10-10 00:00:00,3.005341e+14,2023-10-10 06:00:00,3.005341e+14,2023-10-10 12:00:00,3.005341e+14,2023-10-10 18:00:00,3.005341e+14,2023-10-11 00:00:00,...,3.005341e+14,2023-10-12 18:00:00,3.005341e+14,2023-10-13 00:00:00,3.005341e+14,2023-10-13 06:00:00,3.005341e+14,2023-10-13 12:00:00,300534064920860,2023-10-13 18:00:00
63710,3.005341e+14,2023-10-10 06:00:00,3.005341e+14,2023-10-10 12:00:00,3.005341e+14,2023-10-10 18:00:00,3.005341e+14,2023-10-11 00:00:00,3.005341e+14,2023-10-11 06:00:00,...,3.005341e+14,2023-10-13 00:00:00,3.005341e+14,2023-10-13 06:00:00,3.005341e+14,2023-10-13 12:00:00,3.005341e+14,2023-10-13 18:00:00,300534064920860,2023-10-14 00:00:00
63711,3.005341e+14,2023-10-10 12:00:00,3.005341e+14,2023-10-10 18:00:00,3.005341e+14,2023-10-11 00:00:00,3.005341e+14,2023-10-11 06:00:00,3.005341e+14,2023-10-11 12:00:00,...,3.005341e+14,2023-10-13 06:00:00,3.005341e+14,2023-10-13 12:00:00,3.005341e+14,2023-10-13 18:00:00,3.005341e+14,2023-10-14 00:00:00,300534064920860,2023-10-14 06:00:00
